<a href="https://colab.research.google.com/github/lmmlima/ENV716_EnergyModeling_F2021/blob/main/Lab3/Lab3_PivotTables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3 – More on Filtering Data Sets and Pivot Tables in Python**

Learning goals for Lab 3:

*   Learn how to change column type from character to float/int;
*   Learn how to create pivot tables from a data frame using function pivot_table();
*   Learn how to save your table as an excel file.



If you have not mounted google drive yet please do so by running the code chunk below. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set up working directory.

In [ ]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/')

Before we get started take a look at the Runtime tab and all the ways you can run your notebook code chunks.


*   Run all
*   Run before
*   Run selection
*   Run after


# Import Excel File

Before building the pivot table you need to import the data. Here we will import sheet GEN19 from e-grid to Python.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_gen=pd.read_excel("egrid2019_data.xlsx",sheet_name="GEN19",skiprows=1,keep_default_na=False)
df_gen.head()

,SEQGEN19,YEAR,PSTATABB,PNAME,ORISPL,GENID,NUMBLR,GENSTAT,PRMVR,FUELG1,NAMEPCAP,CFACT,GENNTAN,GENNTOZ,GENERSRC,GENYRONL,GENYRRET
0,1,2019,AK,7-Mile Ridge Wind Project,60814,WT1,0,CN,WT,WND,1.8,,,,,,
1,2,2019,AK,Agrium Kenai Nitrogen Operations,54452,744A,0,OS,GT,NG,2.5,,,,,1977,
2,3,2019,AK,Agrium Kenai Nitrogen Operations,54452,744B,0,OS,GT,NG,2.5,,,,,1977,
3,4,2019,AK,Agrium Kenai Nitrogen Operations,54452,744C,0,OS,GT,NG,2.5,,,,,1977,
4,5,2019,AK,Agrium Kenai Nitrogen Operations,54452,744D,0,OS,GT,NG,2.5,,,,,1977,


In [ ]:
df_gen.columns

Index(['SEQGEN19', 'YEAR', 'PSTATABB', 'PNAME', 'ORISPL', 'GENID', 'NUMBLR',
       'GENSTAT', 'PRMVR', 'FUELG1', 'NAMEPCAP', 'CFACT', 'GENNTAN', 'GENNTOZ',
       'GENERSRC', 'GENYRONL', 'GENYRRET'],
      dtype='object')

# Building a Pivot Table with Python

A pivot table allows you to reshape your original table or data frame. Pandas offers two functions to pivot tables: pivot() and pivot_table(). The pivot_table() is a generalization of pivot() that allows you to aggregate multiple values and create summaries. 

The pivot_table() function takes two main arguments: index and columns. The index will identify which column of the data frame you want to be represented in the rows of the pivot table. If you think about the pivot table in Excel, the elements will put in the Rows category should be the same as you put in index. Columns is whichever element from the data frame you want to appear in the columns of the pivot. 

The documentation for pivot_table() can be found at:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html.

Let’s see how to use pivot_table(). 


Imagine you want to filter only generators online between 2007 and 2018. Let’s create a subset of generators that satisfy that criteria before entering pivot_table().

In [ ]:
df.dtypes

SEQGEN19      int64
YEAR          int64
PSTATABB     object
PNAME        object
ORISPL        int64
GENID        object
NUMBLR        int64
GENSTAT      object
PRMVR        object
FUELG1       object
NAMEPCAP     object
CFACT        object
GENNTAN      object
GENNTOZ      object
GENERSRC     object
GENYRONL    float64
GENYRRET     object
dtype: object

Note GENYRONL is an object so we need to convert to numeric using pd.to_numeric() function.

In [ ]:
df_gen['GENYRONL']=pd.to_numeric(df_gen['GENYRONL'])

In [ ]:
sub_df=df_gen[(df_gen.GENYRONL >= 2007) & (df_gen.GENYRONL <= 2018)]

In [ ]:
sub_df=sub_df.loc[:,['PSTATABB','PNAME','FUELG1','CFACT','GENYRONL']]

In [ ]:
sub_df.head()

,PSTATABB,PNAME,FUELG1,CFACT,GENYRONL
13,AK,Alakanuk,DFO,-0.002,2013.0
18,AK,Allison Creek Hydro,WAT,0.336,2016.0
22,AK,Anchorage 1,NG,0.022,2007.0
24,AK,Anchorage 1,DFO,0,2012.0
30,AK,Angoon,DFO,0.123,2009.0


If you want to fix the decimal points on the GENYRONL columns we can use the to_numeric() conversion again, but specify you want the column to be converted to integer using argument downcast=’signed’. You can use ’signed’ or ‘integer’. 

In [ ]:
sub_df['GENYRONL']=pd.to_numeric(sub_df['GENYRONL'],downcast='signed') #or 'integer'
sub_df.dtypes

PSTATABB    object
PNAME       object
FUELG1      object
CFACT       object
GENYRONL     int16
dtype: object

In [ ]:
sub_df.head()

,PSTATABB,PNAME,FUELG1,CFACT,GENYRONL
13,AK,Alakanuk,DFO,-0.002,2013
18,AK,Allison Creek Hydro,WAT,0.336,2016
22,AK,Anchorage 1,NG,0.022,2007
24,AK,Anchorage 1,DFO,0,2012
30,AK,Angoon,DFO,0.123,2009


To create a pivot table you need to use the pivot_table() function. If you want states in the rows specify index= ‘PSTATABB’ and to get year in the columns specify columns=‘GENYRONL’. 
Since we want to count units that come online each year specify aggfunc=”count”.You can include another argument that says values=’PNAMES’ so the function nows what to count. It could be any column on sub_df assigned to values. We are just making sure the counting only happens once.

In [ ]:
pivot1=sub_df.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",margins=True)
pivot1.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
PSTATABB,,,,,,,,,,,,,
AK,12.0,14.0,13.0,20.0,5.0,21.0,20.0,14.0,23.0,12.0,14.0,3.0,171
AL,1.0,3.0,NaN,6.0,2.0,NaN,3.0,1.0,NaN,1.0,6.0,1.0,24
AR,10.0,NaN,1.0,8.0,NaN,1.0,1.0,NaN,1.0,2.0,3.0,1.0,28
AZ,NaN,6.0,7.0,7.0,33.0,30.0,19.0,8.0,8.0,12.0,11.0,6.0,147
CA,30.0,41.0,47.0,71.0,129.0,164.0,169.0,181.0,179.0,132.0,124.0,88.0,1355
CO,12.0,22.0,11.0,10.0,8.0,19.0,11.0,8.0,14.0,12.0,17.0,16.0,160
CT,2.0,9.0,6.0,23.0,9.0,7.0,4.0,9.0,5.0,16.0,6.0,25.0,121
DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,5
DE,NaN,NaN,NaN,2.0,4.0,4.0,8.0,2.0,3.0,NaN,1.0,2.0,26


If you want to fix the NaN, you can add the fill_value argument to the function as bellow.

In [ ]:
pivot1=sub_df.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",fill_value="",margins=True)
pivot1.head(51)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,All
PSTATABB,,,,,,,,,,,,,
AK,12,14,13,20,5,21,20,14,23,12,14,3,171
AL,1,3,,6,2,,3,1,,1,6,1,24
AR,10,,1,8,,1,1,,1,2,3,1,28
AZ,,6,7,7,33,30,19,8,8,12,11,6,147
CA,30,41,47,71,129,164,169,181,179,132,124,88,1355
CO,12,22,11,10,8,19,11,8,14,12,17,16,160
CT,2,9,6,23,9,7,4,9,5,16,6,25,121
DC,,,,,,,,,3,1,,1,5
DE,,,,2,4,4,8,2,3,,1,2,26


If you want to change the name of the summary columns you can add another argument margins_name="Total".

In [ ]:
pivot1=sub_df.pivot_table(values="PNAME",index="PSTATABB",columns="GENYRONL",aggfunc="count",margins_name="Total",fill_value="",margins=True)
pivot1.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total
PSTATABB,,,,,,,,,,,,,
AK,12,14,13,20,5,21,20,14,23,12,14,3,171
AL,1,3,,6,2,,3,1,,1,6,1,24
AR,10,,1,8,,1,1,,1,2,3,1,28
AZ,,6,7,7,33,30,19,8,8,12,11,6,147
CA,30,41,47,71,129,164,169,181,179,132,124,88,1355
CO,12,22,11,10,8,19,11,8,14,12,17,16,160
CT,2,9,6,23,9,7,4,9,5,16,6,25,121
DC,,,,,,,,,3,1,,1,5
DE,,,,2,4,4,8,2,3,,1,2,26


You can export your pivot table to Excel using the command .to_excel() as below. And a excel file will be created on your working directory. If you want to specify a name for the sheet use teh argumento sheet_name.

In [ ]:
pivot1.to_excel("output1.xlsx",sheet_name="PIVOT1")

Where was you file stored?? Since you did not specify the path, the code you store it at your current working directory.

One of the confusing points with the pivot_table() is the use of columns and values. Remember, columns provide a way to segment the values you care about. The aggregation functions are applied to the values you list. Let’s see a couple more examples.

Let’s create a table that shows average capacity factor by state of the generators that came online in 2007, 2008, …, 2018. You need to change values to ‘CFACT’ and aggfunc to “mean”.

In [ ]:
sub_df.dtypes

PSTATABB    object
PNAME       object
FUELG1      object
CFACT       object
GENYRONL     int16
dtype: object

In [ ]:
sub_df['CFACT']=pd.to_numeric(sub_df['CFACT'])

In [ ]:
sub_df.dtypes

PSTATABB     object
PNAME        object
FUELG1       object
CFACT       float64
GENYRONL      int16
dtype: object

In [ ]:
pivot2=sub_df.pivot_table(values="CFACT",index="PSTATABB",columns="GENYRONL",aggfunc="mean",margins_name="Av CFAC",fill_value="",margins=True)
pivot2.head(10)

GENYRONL,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Av CFAC
PSTATABB,,,,,,,,,,,,,
AK,0.228583,0.170786,0.195923,0.12935,0.1344,0.173619,0.23675,0.245643,0.282727,0.141636,0.167231,0.154667,0.197476
AL,0.988,0.688,,0.544,0.584,,0.664,0.722,,0.228,0.213667,0.21,0.496583
AR,0.2777,,0.481,0.681625,,0.753,0.024,,0.248,0.5165,0.385667,0.229,0.434107
AZ,,0.251333,0.321286,0.231571,0.161156,0.2839,0.32,0.235875,0.28425,0.275667,0.186455,0.1845,0.244925
CA,0.1945,0.3365,0.253444,0.327721,0.132898,0.303411,0.249679,0.289326,0.201713,0.305902,0.352598,0.342908,0.270319
CO,0.184083,0.158727,0.190364,0.2258,0.26925,0.352211,0.331273,0.330625,0.362429,0.268333,0.248294,0.2466,0.26039
CT,0.2985,0.411333,0.531667,0.101652,0.289222,0.300857,0.554667,0.229222,0.503,0.429813,0.128833,0.22008,0.28275
DC,,,,,,,,,0.446,0.676,,0.868,0.5764
DE,,,,0.48,0.14975,0.48875,0.645,0.519,0.406,,0.198,0.1545,0.439885


Now let’s create a table that count generators again but let’s include another layer of discretization, unit primary fuel. Add another element ‘FUELG1’ to the index argument as show below.



In [ ]:
pivot3=sub_df.pivot_table(values="PNAME",index=["PSTATABB","FUELG1"],columns="GENYRONL",aggfunc="count",margins_name="Total",fill_value="",margins=True)
pivot3.head(10)

GENYRONL        2007 2008 2009 2010 2011 2012  ... 2014 2015 2016 2017 2018 Total
PSTATABB FUELG1                                ...                               
AK       DFO       9    9    9   20    3   12  ...    7    8   11   11    3   111
         LFG                                4  ...                              5
         MWH                                1  ...         2                    3
         NG        1    3    1                 ...    3   10         3         27
         WAT            1    1                 ...    4    2    1               9
         WND            1    2         2    4  ...         1                   14
         WO        2                           ...                              2
AL       BLQ       1                           ...    1              1          3
         LFG                      1    1       ...                              5
         MWH                                   ...                   1          1

[10 rows x 13 columns]

# Exercise H4 - 1

1) Using the spreadsheet PLNT19, build a table that shows for each type of fuel and for each state the following information :

*   Number of plants, 
*   Total nameplate capacity in MW
*   Average capacity factor

To determine the fuel used by the plant, lets refer to: Plant primary coal/oil/gas/ other fossil fuel category (Column Y in the PLNT19 tab )


In [ ]:
df_plt=pd.read_excel("egrid2019_data.xlsx",sheet_name="PLNT19",skiprows=1,keep_default_na=False)
df_plt.head()

In [ ]:
df_plt.columns

In [ ]:
df_plt['NAMEPCAP']=pd.to_numeric(df_plt['NAMEPCAP'])
df_plt['CAPFAC']=pd.to_numeric(df_plt['CAPFAC'])

In [ ]:
pivot4=df_plt.pivot_table(values=["PNAME","NAMEPCAP","CAPFAC"],index="PSTATABB",columns="PLFUELCT",aggfunc={"PNAME":"count","NAMEPCAP":np.sum,"CAPFAC":np.mean},margins_name="Total",fill_value="",margins=True)
pivot4.head(10)

# Exercise H4 - 2

2) Build a table that shows for each state, and for each type of Fossil fuel information on:

*   Number of plants
*   Total nameplate capacity in MW
*   Average capacity factor

In [ ]:
#your code

# Exercise H4 - 3

3) Build a table that shows for each state the amount of installed capacity that came online for solar and wind since year 2000. Show :
*   number of generators 
*   total nameplate capacity in MW



In [ ]:
#your code